In [1]:
library(select)
library(tidyverse)

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ dplyr::select() masks select::select()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1   dplyr_1.1.4    
 [5] purrr_1.0.2     readr_2.1.5     tidyr_1.3.0     tibble_3.2.1   
 [9] ggplot2_3.4.4   tidyverse_2.0.0 select_1.6.4   

loaded via a namespace (and not attach

In [3]:
filter_run_data <- function(run_data,gene_order,sample_order){
    final_run_data<-run_data
    final_run_data$M$M$missense<-run_data$M$M$missense[gene_order,sample_order]
    final_run_data$M$M$truncating<-run_data$M$M$truncating[gene_order,sample_order]
    final_run_data$alteration.class<- run_data$alteration.class[gene_order]
    final_run_data$sample.class<-run_data$sample.class[sample_order]
    final_run_data$M$tmb$missense<-run_data$M$tmb$missense[sample_order,]
    final_run_data$M$tmb$truncating<-run_data$M$tmb$truncating[sample_order,]
    return(final_run_data)
}

# Note
- Similar way to do for TCGA pan-cancer oncogenic and silent mutations

# LUAD

In [4]:
run_data <- readRDS('/mnt/ndata/arvind/co_mutation_project/data/processed/gams/pan_can_tcga_run_data.rds')

In [5]:
sample_covariates<-run_data$sample.class

In [6]:
luad_samples <- names(sample_covariates[(which(sample_covariates %in% c('LUAD')))])

In [7]:
data<- filter_run_data(run_data = run_data,gene_order = names(run_data$alteration.class),sample_order = luad_samples)

In [8]:
str(run_data)

List of 3
 $ M               :List of 2
  ..$ M  :List of 2
  .. ..$ missense  : num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. ..$ truncating: num [1:396, 1:9082] 0 0 0 0 0 0 0 0 0 0 ...
  .. .. ..- attr(*, "dimnames")=List of 2
  .. .. .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
  .. .. .. ..$ : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  ..$ tmb:List of 2
  .. ..$ missense  :'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-AB-2805-03" "TCGA-AB-2806-03" ...
  .. .. ..$ mutation: num [1:9082] 56 47 40 422 214 588 14 8 19 8 ...
  .. ..$ truncating:'data.frame':	9082 obs. of  2 variables:
  .. .. ..$ sample  : chr [1:9082] "TCGA-02-0047-01" "TCGA-02-0055-01" "TCGA-

In [9]:
gam<-data$M$M[[1]]+data$M$M[[2]][rownames(data$M$M[[1]]),colnames(data$M$M[[1]])]

In [10]:
gam[gam>1]<-1

In [13]:
gam_list <- list('gam'=t(gam),'alt'=data$alteration_covariates,'samples'=data$sample_covariates)        

In [14]:
str(gam_list)

List of 3
 $ gam    : num [1:502, 1:396] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:502] "TCGA-05-4244-01" "TCGA-05-4249-01" "TCGA-05-4250-01" "TCGA-05-4382-01" ...
  .. ..$ : chr [1:396] "AKT1" "ALK" "APC" "AR" ...
 $ alt    : NULL
 $ samples: NULL


In [ ]:
select_run<-select::select(M = gam_list$gam,
                           sample.class = gam_list$samples,
                           alteration.class = gam_list$alt,
                           folder ='.',
                           r.seed = 110,
                           n.cores = 30,
                           n.permut = 1000,
                           min.feature.support = 1,
                           save.intermediate.files=FALSE,
                           randomization.switch.threshold =20,
                           verbose = TRUE,
                           FDR.thresh = 0.25)

In [22]:
select_run %>% filter(wMI_p.value_FDR) %>% count(direction)

direction,n
<chr>,<int>
CO,117
ME,79


In [14]:
saveRDS(select_run,'../results/luad_all_results.rds')

## Sampling Run

In [15]:
sample_list1 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad.rds')
sample_list2 <- readRDS('/mnt/ptemp/arvind/tool_comaprision/data/sampling_list_luad_2.rds')

In [16]:
str(c(sample_list1[11:15],sample_list2[11:15]))

List of 10
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-78-7539-01" "TCGA-99-8033-01" "TCGA-73-4670-01" "TCGA-75-5146-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-75-6214-01" "TCGA-64-5775-01" "TCGA-97-A4M3-01" "TCGA-93-A4JO-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-64-1679-01" "TCGA-64-1676-01" "TCGA-55-8505-01" "TCGA-97-8547-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-55-8204-01" "TCGA-55-8094-01" "TCGA-49-AARE-01" "TCGA-44-A47G-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-05-4422-01" "TCGA-49-AAR9-01" "TCGA-05-4417-01" "TCGA-55-6984-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-91-6848-01" "TCGA-55-6971-01" "TCGA-S2-AA1A-01" "TCGA-86-

In [17]:
sample_list<-c(sample_list1[11:15],sample_list2[11:15])

In [18]:
str(sample_list)

List of 10
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-78-7539-01" "TCGA-99-8033-01" "TCGA-73-4670-01" "TCGA-75-5146-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-75-6214-01" "TCGA-64-5775-01" "TCGA-97-A4M3-01" "TCGA-93-A4JO-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-64-1679-01" "TCGA-64-1676-01" "TCGA-55-8505-01" "TCGA-97-8547-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-55-8204-01" "TCGA-55-8094-01" "TCGA-49-AARE-01" "TCGA-44-A47G-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-05-4422-01" "TCGA-49-AAR9-01" "TCGA-05-4417-01" "TCGA-55-6984-01" ...
 $ : Named chr [1:401] "LUAD" "LUAD" "LUAD" "LUAD" ...
  ..- attr(*, "names")= chr [1:401] "TCGA-91-6848-01" "TCGA-55-6971-01" "TCGA-S2-AA1A-01" "TCGA-86-

In [ ]:
result_vec<-list()
for(i in c(1:10)){
    order <- names(sample_list[[i]])
    data<- filter_run_data(run_data = run_data,gene_order = names(run_data$alteration.class),sample_order = order)
    sample_covariates<-run_data$sample.class
    sample_covariates<-sample_covariates[[order]]
    data <- list('M'=data,'sample.class' = sample_covariates,'alteration.class' = alteration_covariates)
    gam<-data$M$M[[1]]+data$M$M[[2]][rownames(data$M$M[[1]]),colnames(data$M$M[[1]])]
    gam[gam>1]<-1
    gam_list <- list('gam'=t(gam),'alt'=data$alteration_covariates,'samples'=data$sample_covariates)
    select_run<-select::select(M = gam_list$gam,
                           sample.class = gam_list$samples,
                           alteration.class = gam_list$alt,
                           folder ='.',
                           r.seed = 110,
                           n.cores = 30,
                           n.permut = 1000,
                           min.feature.support = 1,
                           save.intermediate.files=FALSE,
                           randomization.switch.threshold =20,
                           verbose = TRUE,
                           FDR.thresh = 0.25)
    result_vec[[i]]<-select_run
    saveRDS(select_run,file=paste('/mnt/ptemp/arvind/tool_comaprision/results/select/sampling_run_result_80_',i,'.rds',sep=""))    
    rm(select_run)
}